In [1]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
from matplotlib import pyplot as plt


In [2]:
sf_crime = pd.read_csv('resources/Police_Department_Incident_Reports__2018_to_Present (1)\Police_Department_Incident_Reports__2018_to_Present (1).csv')
sf_crime

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745681,2023/03/22 01:30:00 PM,2023/03/22,13:30,2023,Wednesday,2023/03/22 01:35:00 PM,125685303401,1256853,230201949,230811847.0,...,-122.406336,POINT (-122.40633623830558 37.75300402962228),54.0,NaN,NaN,NaN,3.0,NaN,2.0,3.0
745682,2023/03/22 12:13:00 AM,2023/03/22,00:13,2023,Wednesday,2023/03/22 12:13:00 AM,125665206371,1256652,230200838,230810020.0,...,-122.440624,POINT (-122.44062361999508 37.78899525864335),102.0,NaN,NaN,NaN,NaN,NaN,6.0,4.0
745683,2023/03/21 09:19:00 PM,2023/03/21,21:19,2023,Tuesday,2023/03/21 10:16:00 PM,125664272000,1256642,230200800,230803734.0,...,-122.452665,POINT (-122.45266472219633 37.70880633652071),66.0,NaN,NaN,NaN,NaN,NaN,1.0,9.0
745684,2023/03/22 03:28:00 PM,2023/03/22,15:28,2023,Wednesday,2023/03/22 05:51:00 PM,125687306303,1256873,230202486,230812680.0,...,-122.405574,POINT (-122.40557374633903 37.790565236529574),19.0,NaN,NaN,NaN,NaN,NaN,3.0,6.0


In [3]:
num_nan_val = sf_crime.isna().sum()
print(num_nan_val)

Incident Datetime                                            0
Incident Date                                                0
Incident Time                                                0
Incident Year                                                0
Incident Day of Week                                         0
Report Datetime                                              0
Row ID                                                       0
Incident ID                                                  0
Incident Number                                              0
CAD Number                                              168715
Report Type Code                                             0
Report Type Description                                      0
Filed Online                                            594454
Incident Code                                                0
Incident Category                                          658
Incident Subcategory                                   

In [4]:
sf_crime = sf_crime.drop(['ESNCAG - Boundary File', 'Central Market/Tenderloin Boundary Polygon - Updated', 'Civic Center Harm Reduction Project Boundary', 'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas', 'Current Supervisor Districts'], axis=1)


In [5]:
sf_crime.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,CNN,Police District,Analysis Neighborhood,Supervisor District,Supervisor District 2012,Latitude,Longitude,Point,Neighborhoods,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,...,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,...,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,...,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,...,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,...,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sf_crime= sf_crime.drop(['CAD Number', 'CNN', 'Analysis Neighborhood', 'Supervisor District', 'Supervisor District 2012', 'Latitude', 'Longitude', 'Point', 'Neighborhoods'], axis=1)
sf_crime.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,Police District,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,Mission,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN


In [7]:
sf_crime = sf_crime.drop(['Incident Datetime'], axis=1)

In [8]:
sf_crime.head()

,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,Police District,Current Police Districts
0,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
1,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,Mission,NaN
2,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
3,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN
4,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,Out of SF,NaN


In [9]:
sf_crime = sf_crime.drop(['Intersection', 'Current Police Districts'], axis = 1)

In [10]:
sf_crime = sf_crime.dropna()
sf_crime.head()

,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Police District
1,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Mission
5,2023/02/15,03:00,2023,Wednesday,2023/03/11 04:55:00 PM,125378606372,1253786,236046123,II,Coplogic Initial,True,6372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Mission
6,2023/03/11,12:30,2023,Saturday,2023/03/12 04:15:00 PM,125381606244,1253816,236046004,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
7,2023/03/13,11:26,2023,Monday,2023/03/13 01:37:00 PM,125419506244,1254195,236046850,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
8,2023/03/11,15:00,2023,Saturday,2023/03/13 08:29:00 AM,125420606244,1254206,236045937,IS,Coplogic Supplement,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central


In [11]:
sf_crime = sf_crime.drop(['Incident Date', 'Report Datetime'], axis = 1)
sf_crime.head()

,Incident Time,Incident Year,Incident Day of Week,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Police District
1,05:02,2023,Wednesday,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Mission
5,03:00,2023,Wednesday,125378606372,1253786,236046123,II,Coplogic Initial,True,6372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Mission
6,12:30,2023,Saturday,125381606244,1253816,236046004,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
7,11:26,2023,Monday,125419506244,1254195,236046850,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
8,15:00,2023,Saturday,125420606244,1254206,236045937,IS,Coplogic Supplement,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central


In [12]:
from sklearn.preprocessing import LabelEncoder
day_dict = {
    'Sunday': 1,
    'Monday': 2,
    'Tuesday': 3,
    'Wednesday': 4,
    'Thursday': 5,
    'Friday': 6,
    'Saturday': 7
}
sf_crime['Incident Day of Week'] = sf_crime['Incident Day of Week'].map(day_dict)
sf_crime.head()

,Incident Time,Incident Year,Incident Day of Week,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Police District
1,05:02,2023,4,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Mission
5,03:00,2023,4,125378606372,1253786,236046123,II,Coplogic Initial,True,6372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Mission
6,12:30,2023,7,125381606244,1253816,236046004,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
7,11:26,2023,2,125419506244,1254195,236046850,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
8,15:00,2023,7,125420606244,1254206,236045937,IS,Coplogic Supplement,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central


In [13]:
sf_crime.rename(columns = {'Incident Time': 'incident_time', 'Incident Year': 'incident_year', 'Incident Day of Week': 'incident_dow', 'Row ID': 'row_id', 'Incident ID': 'incident_id', 'Incident Number': 'incident_number', 'Report Type Code': 'report_type_code', 'Report Type Description': 'report_type_desc', 'Filed Online': 'filed_online', 'Incident Code': 'incident_code', 'Incident Category': 'incident_cat', 'Incident Subcategory': 'incident_sub', 'Incident Description': 'incident_desc', 'Resolution': 'resolution', 'Police District': 'police_district'}, inplace=True)
sf_crime.head()

,incident_time,incident_year,incident_dow,row_id,incident_id,incident_number,report_type_code,report_type_desc,filed_online,incident_code,incident_cat,incident_sub,incident_desc,resolution,police_district
1,05:02,2023,4,125379506374,1253795,236046151,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,Mission
5,03:00,2023,4,125378606372,1253786,236046123,II,Coplogic Initial,True,6372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,Mission
6,12:30,2023,7,125381606244,1253816,236046004,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
7,11:26,2023,2,125419506244,1254195,236046850,II,Coplogic Initial,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central
8,15:00,2023,7,125420606244,1254206,236045937,IS,Coplogic Supplement,True,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,Central


In [15]:
dist_encoder = LabelEncoder()
y_numeric=dist_encoder.fit_transform
sf_crime['police_district'] = dist_encoder.fit_transform(sf_crime['police_district'])

In [16]:
res_encoder = LabelEncoder()
sf_crime['resolution'] = res_encoder.fit_transform(sf_crime['resolution'])

In [17]:
incid_desc_encoder = LabelEncoder()
sf_crime['incident_desc'] = incid_desc_encoder.fit_transform(sf_crime['incident_desc'])

In [18]:
incid_sub_encoder = LabelEncoder()
sf_crime['incident_sub'] = incid_sub_encoder.fit_transform(sf_crime['incident_sub'])

In [19]:
incid_cat_encoder = LabelEncoder()
sf_crime['incident_cat'] = incid_cat_encoder.fit_transform(sf_crime['incident_cat'])

In [20]:
incid_code_encoder = LabelEncoder()
sf_crime['incident_code'] = incid_code_encoder.fit_transform(sf_crime['incident_code'])

In [21]:
online_encoder = LabelEncoder()
sf_crime['filed_online'] = online_encoder.fit_transform(sf_crime['filed_online'])

In [22]:
report_type_desc_encoder = LabelEncoder()
sf_crime['report_type_desc'] = report_type_desc_encoder.fit_transform(sf_crime['report_type_desc'])

In [23]:
report_type_code_encoder = LabelEncoder()
sf_crime['report_type_code'] = report_type_code_encoder.fit_transform(sf_crime['report_type_code'])

In [24]:
incid_num_encoder = LabelEncoder()
sf_crime['incident_number'] = incid_num_encoder.fit_transform(sf_crime['incident_number'])

In [25]:
incid_id_encoder = LabelEncoder()
sf_crime['incident_id'] = incid_id_encoder.fit_transform(sf_crime['incident_id'])

In [26]:
row_id_encoder = LabelEncoder()
sf_crime['row_id'] = row_id_encoder.fit_transform(sf_crime['row_id'])

In [27]:
year_encoder = LabelEncoder()
sf_crime['incident_year'] = year_encoder.fit_transform(sf_crime['incident_year'])

In [28]:
res_encoder = LabelEncoder()
sf_crime['resolution'] = res_encoder.fit_transform(sf_crime['resolution'])

In [29]:
time_encoder = LabelEncoder()
sf_crime['incident_time'] = time_encoder.fit_transform(sf_crime['incident_time'])

In [30]:
sf_crime.head()

,incident_time,incident_year,incident_dow,row_id,incident_id,incident_number,report_type_code,report_type_desc,filed_online,incident_code,incident_cat,incident_sub,incident_desc,resolution,police_district
1,302,5,4,141716,141716,134948,0,0,0,9,4,6,17,0,3
5,180,5,4,141707,141707,134945,0,0,0,8,4,6,16,0,3
6,749,5,7,141736,141736,134933,0,0,0,3,4,4,12,0,1
7,685,5,2,141834,141834,135018,0,0,0,3,4,4,12,0,1
8,899,5,7,141845,141845,134926,1,1,0,3,4,4,12,0,1


In [31]:
X = sf_crime.drop(columns=['police_district'])
y = sf_crime['police_district']

In [32]:
X = sf_crime[['incident_time', 'incident_year', 'incident_dow', 'row_id', 'incident_id', 'incident_number', 'report_type_code', 'report_type_desc', 'filed_online', 'incident_code', 'incident_cat', 'incident_sub', 'incident_desc', 'resolution', 'police_district']]
y = sf_crime['police_district']


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [35]:
predictions = rf_model.predict(X_test)

In [36]:
accuracy = rf_model.score(X_test, y_test)
print("Prediction accuracy:", accuracy)

Prediction accuracy: 0.9945779746751744


In [37]:
selected_district = y_test[predictions == 1]

In [38]:
selected_district_name = dist_encoder.inverse_transform(selected_district)
print(selected_district)

532375    1
493064    1
269800    1
710312    1
581083    1
         ..
59926     1
581133    1
114962    1
646501    1
335150    1
Name: police_district, Length: 5936, dtype: int32


In [39]:

# selected_district = y_test[predictions ==1].iloc[0]
print('Best District for a new business:', selected_district)

Best District for a new business: 532375    1
493064    1
269800    1
710312    1
581083    1
         ..
59926     1
581133    1
114962    1
646501    1
335150    1
Name: police_district, Length: 5936, dtype: int32
